In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as U
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import random

### Define the Neural Network

Simple network with two layers.
We want to be able to set the weight of the network manually, and get them in the form of a flat vector.

In [2]:
class NeuralNet(nn.Module):
    def __init__(self, in_dim=2, out_dim=2):
        super(NeuralNet, self).__init__()
        
        self.layer1=nn.Linear(in_features=in_dim, out_features=4)
        self.layer2=nn.Linear(in_features=4, out_features=out_dim)

        self.weights_initialization()
    
    def weights_initialization(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)  # This is the default in PyTorch
                nn.init.constant_(module.bias, 0)

    def forward(self, x):
        out = self.layer1(x)
        out = F.relu(out)
        out = self.layer2(out)
        return out
    
    def get_flat_params(self):
        return U.parameters_to_vector(self.parameters())
    
    def set_flat_params(self, flat_params):
        U.vector_to_parameters(flat_params, self.parameters())

### Load Data

In [3]:
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

### Evolutionary Algorithm

The parameters are the genes.
The flat vector of parameters is the chromosome.
The population is the different random combinations of parameters.

In [4]:
def generate_population(size, chromosome_length):
    """ Generate random population """
    return [torch.randn(chromosome_length) for _ in range(size)]

def fitness(chromosome, model):
    """ Algorithm's fitness function """
    model.set_flat_params(chromosome)   # Update model's parameters with the chromosome
    with torch.no_grad():
        outputs = model(X_test)
        pred = outputs.argmax(dim=1)
        accuracy = (pred == y_test).float().mean().item()
    return accuracy

def selection(population, fitness_scores):
    """ Roulette Wheel Selection """
    total_fitness = sum(fitness_scores)

    if total_fitness == 0:  # If it's 0 we get an uniform distribution
        selection_probs = [1 / len(fitness_scores)] * len(fitness_scores)
    else:
        selection_probs = [f / total_fitness for f in fitness_scores]
    
    parent1 = population[random.choices(range(len(population)), selection_probs)[0]]
    parent2 = population[random.choices(range(len(population)), selection_probs)[0]]
    return parent1, parent2

def crossover(parent1, parent2):
    """Two-Point Crossover"""
    p1, p2 = sorted(random.sample(range(1, len(parent1)), 2))
    offspring1 = torch.cat([parent1[:p1], parent2[p1:p2], parent1[p2:]])
    offspring2 = torch.cat([parent2[:p1], parent1[p1:p2], parent2[p2:]])
    return offspring1, offspring2

def mutate(chromosome, mutation_rate=0.05, mutation_strength=0.1):
    """
        Mutate a small subset of the genes of the chromosome.

        Args:
            chromosome (tensor): Chromosome to mutate.
            mutation_rate (float): Probability of mutating each gene.
            mutation_strength (float): How much noise is aggregated to mutated genes.
    """
    mutated = chromosome.clone()
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            mutated[i] += torch.randn(1).item() * mutation_strength
    return mutated


In [5]:
# Initialize model and get flat parameters
model = NeuralNet()
params_vector = model.get_flat_params()
print(f"Flat Params: {params_vector}\n")

population = generate_population(1, len(params_vector))
fitness_scores = [fitness(individual, model) for individual in population]
parent1, parent2 = selection(population, fitness_scores)
offspring1, offspring2 = crossover(parent1, parent2)
mutated_offspring1 = mutate(offspring1)

print(f"Population Sample: {population[0]}\n")
print(f"Fitness Score Sample: {fitness_scores[0]}\n")
print(f"Selection Result: {parent1, parent2}\n")
print(f"Crossover Result: {offspring1, offspring2}\n")
print(f"Mutated Offspring 1: {mutated_offspring1}")


Flat Params: tensor([-0.8489, -0.8841, -0.2412,  0.4324,  0.8400, -0.8580,  0.6488, -0.5923,
         0.0000,  0.0000,  0.0000,  0.0000,  0.1888, -0.6050,  0.6828, -0.3111,
        -0.4196, -0.2563,  0.5051, -0.0666,  0.0000,  0.0000],
       grad_fn=<CatBackward0>)

Population Sample: tensor([-2.2779,  1.0271,  0.9689, -1.2743, -1.1449,  2.2744, -0.2585, -0.2658,
         0.7961, -0.3824, -0.9746,  0.2600,  1.1276,  0.1932, -0.7415, -1.1081,
        -0.2385,  0.4459, -0.9192, -0.9908,  0.5050, -0.1834])

Fitness Score Sample: 0.5

Selection Result: (tensor([-2.2779,  1.0271,  0.9689, -1.2743, -1.1449,  2.2744, -0.2585, -0.2658,
         0.7961, -0.3824, -0.9746,  0.2600,  1.1276,  0.1932, -0.7415, -1.1081,
        -0.2385,  0.4459, -0.9192, -0.9908,  0.5050, -0.1834]), tensor([-2.2779,  1.0271,  0.9689, -1.2743, -1.1449,  2.2744, -0.2585, -0.2658,
         0.7961, -0.3824, -0.9746,  0.2600,  1.1276,  0.1932, -0.7415, -1.1081,
        -0.2385,  0.4459, -0.9192, -0.9908,  0.5050, -0.183

In [6]:
def genetic_algorithm(population_size, chromosome_length, n_generations, model):
    population = generate_population(population_size, chromosome_length)    # Initialize population
    best, best_fitness = 0, 0   # Keeping track of best result
    
    # Main loop
    for generation in range(n_generations):
        fitness_scores = [fitness(individual, model) for individual in population]  # Calculate fitness scores
        # Search for the best chromosome
        for i in range(population_size):
            if fitness_scores[i] > best_fitness:
                best = population[i]
                best_fitness = fitness_scores[i]
        
        # Create new generation
        new_gen = []
        while len(new_gen) < population_size:
            parent1, parent2 = selection(population, fitness_scores)
            offspring1, offspring2 = crossover(parent1, parent2)
            child1 = mutate(offspring1)
            child2 = mutate(offspring2)

            new_gen.extend([child1, child2])

        population = new_gen    # Replace popultaion with new generation

    return best, best_fitness


### Train the Neural Network

In [7]:
def evaluate(X, y, model):
    with torch.no_grad():
        outputs = model(X_test)
        preds = outputs.argmax(dim=1)
        acc = (preds == y_test).float().mean().item()
    return acc

##### Without Training

In [8]:
# Initialize model
model = NeuralNet()
params_vector = model.get_flat_params()

# Configuration
POPULATION_SIZE = 100
CHROMOSOME_LENGTH = len(params_vector)
NUM_GENERATIONS = 100

acc = evaluate(X_test, y_test, model)
print(f"\nFinal accuracy of best model: {acc:.4f}")


Final accuracy of best model: 0.5050


In [9]:
best_chromosome, score = genetic_algorithm(POPULATION_SIZE, CHROMOSOME_LENGTH, NUM_GENERATIONS, model)

# Set best individual's params in the model
model.set_flat_params(best_chromosome)

# Evaluate
acc = evaluate(X_test, y_test, model)
print(f"\nFinal accuracy of best model: {acc:.4f}")


Final accuracy of best model: 0.9700
